In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from nuclei_segmentation import my_cellpose
from nuclei_segmentation.quality_control.model_tester import ModelTester

GPU activated: True


In [2]:
# v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

img_path = (
        v.data_path
        + 'SegQA/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz'
)



Grid search into preprocessing steps 

In [3]:
from itertools import permutations, chain
import numpy as np

# Cellpose params
steps_type = ['3D', '2D']
steps_thr = list(np.round(np.linspace(.2, .6, 6), 1))
steps_prob_thr = list(np.round(np.linspace(0, .4, 4), 1))
steps_flow_thr = list(np.round(np.linspace(.2, .6, 4), 1))

# Preprocessing steps
normalization_methods = ['norm_percentile']
pre_steps_top = ['isotropy']
pre_steps_bottom = [
    'bilateral',
    'anisodiff'
]

pre_steps_permutations = list(chain(*[
    permutations(pre_steps_bottom, r)
    for r in range(1, len(pre_steps_bottom) + 1)
])) + [()]

# Postprocessing steps
post_steps_2d = [
    'merge_connected_components',
    'merge_graph',
    'clean_boundaries_opening',
] + [()]

post_steps_3d = [
    'split',
    'clean_boundaries_opening',
    'clean_boundaries_closing',
]

# Generate permutations of post-processing steps for 3D, excluding combinations with both 'clean_boundaries_opening' and 'clean_boundaries_closing'
post_steps_permutations_3d = [
    p for p in chain(*[
        permutations(post_steps_3d, r)
        for r in range(1, len(post_steps_3d) + 1)
    ]) 
    if not ('clean_boundaries_opening' in p and 'clean_boundaries_closing' in p)
] + [()]

pipelines_dict = {}

# Create pipelines for each type (2D/3D)
for step in steps_type:
    # Apply threshold only for 2D
    if step == '2D':
        for thr in steps_thr:
            for flow in steps_flow_thr:
                for norm_method in normalization_methods:
                    for i, pre_steps in enumerate(pre_steps_permutations):
                        for j, post_steps in enumerate(post_steps_2d):
                            pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                            pipeline = pipeline_pre + [post_steps]

                            # Include threshold for 2D pipelines
                            name = f"pipeline_{step}_thr_{thr}_flow_{flow}_{i}_{j}"
                            pipelines_dict[name] = {
                                'pipeline': pipeline,
                                'type': step,
                                'threshold': thr
                            }
    else:  # For 3D pipelines, ignore thresholds
        for norm_method in normalization_methods:
            for prob_t in steps_prob_thr:
                for flow in steps_flow_thr:
                    for i, pre_steps in enumerate(pre_steps_permutations):
                        for j, post_steps in enumerate(post_steps_permutations_3d):
                            pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                            pipeline = pipeline_pre + list(post_steps)

                            # No threshold for 3D pipelines
                            name = f"pipeline_{step}_prob_{prob_t}_flow_{flow}_{i}_{j}"
                            pipelines_dict[name] = {
                                'pipeline': pipeline,
                                'type': step,
                                'prob_threshold': prob_t
                            }

print('Total pipelines:', len(pipelines_dict))


Total pipelines: 1040


In [4]:
pipelines_dict

{'pipeline_3D_prob_0.0_flow_0.2_0_0': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'split'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_1': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_boundaries_opening'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_2': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_boundaries_closing'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_3': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'split',
   'clean_boundaries_opening'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_4': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'split',
   'clean_boundaries_closing'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_5': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_bou

In [5]:
model = my_cellpose.load_model('nuclei')
tester = ModelTester(model)

Loading model: nuclei


/home/imarcoss/mambaforge/envs/cellpose/lib/python3.11/site-packages/cellpose/resnet_torch.py:276: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(file

In [6]:
img_path = (
        v.data_path
        + 'Gr4/RawImages/Nuclei/20191111_E1_DAPI_decon_0.5.nii.gz'
)

pipelines_dict = {
    'pipeline_3D_prob_0.4_flow_0.2_0_1': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_boundaries_opening'],
  'type': '3D',
  'prob_threshold': 0.4},
}

for name, config in pipelines_dict.items():
    print(f"\nTesting {name} pipeline")

    pipeline = config['pipeline']
    type = config['type']
    thr = config['threshold'] if 'threshold' in config else None
    prob_thr = config['prob_threshold'] if 'prob_threshold' in config else .0
    flow_threshold = config['flow_threshold'] if 'flow_threshold' in config else .4

    tester.run(
        img_path, pipeline,
        type=type, stitch_threshold=thr,
        test_name=name, cellprob_threshold=prob_thr,
        flow_threshold=flow_threshold,
        verbose=1
    )


Testing pipeline_3D_prob_0.4_flow_0.2_0_1 pipeline
Running test: pipeline_3D_prob_0.4_flow_0.2_0_1
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/SegQA/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz
Running step: isotropy
	Image resolution: 
	X: 0.9851199984550476 um/px
	Y: 0.9851199984550476 um/px
	Z: 0.4946500062942505 um/px
	Resampling factor: 0.5021215677261353
	Original shape: (40, 110, 140)
	Resampled shape: (20, 110, 140)
Running step: norm_percentile
Running step: bilateral

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/SegQA/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop_pipeline_3D_prob_0.4_flow_0.2_0_1_filtered.nii.gz
Image shape: (20, 110, 140)
	Diameter: 17
	Channels: [0, 0]
	Anisotropy: 1
	Do 3D: True
	Stitch threshold: None
	Cellprob threshold: 0.4
	Flow threshold: 0.4
Masks shape: (20, 110, 140)
Reconstructing image: 1.9915496110916138
Running